In [13]:
#import all the necessary packages

import pandas as pd
import random

import nltk
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk import stem
stemmer = stem.PorterStemmer()
from nltk import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
import string
punct = list(string.punctuation)
from collections import Counter
import requests
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
import numpy as np
import praw
import datetime
import json
import re

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aoife\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aoife\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
import PyPDF2

In [15]:
# open the PDF file
policy = open(r'C:\Users\aoife\Documents\Capstone\Capstone Quant Policies\LIS Disability.pdf', 'rb')

# create a PDF reader object
pdf_reader = PyPDF2.PdfReader(policy)

In [16]:
# read the text from each page
text = ''
for page_num in range(len(pdf_reader.pages)):
    page = pdf_reader.pages[page_num]
    text += page.extract_text()

# close the PDF file
policy.close()

In [17]:
# split the text into sentences
sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)

# create a dataframe with the sentences as rows
df = pd.DataFrame({'Sentence': sentences})

In [18]:
df

,Sentence
0,\n \nPage 1 of 13 \n \nDisability Policy \n ...
1,It outlines the School’s duties and the option...
2,This policy should be read in conjunction with...
3,\n \nScope \n \nThis policy applies to all L...
4,"Where relevant, the policy also applies to pro..."
...,...
145,"Clarified the role of the \nEquality, Diversit..."
146,Included context for \nhow the policy was deve...
147,E xpanded detail \non reasonable adjustments f...
148,\n3 Hannah Kohler (Director of \nAdmission...


In [19]:
#check to see if any rows are empty
rows_with_nan = df[df.isna().any(axis=1)]

len(rows_with_nan)

0

In [20]:
#check the length of the dataframe
len(df)

150

In [21]:
# convert each row in the dataframe to a string and then convert the datadrame to a list of strings
p1 = df.apply(lambda row: ' '.join(row.astype(str)), axis=1).tolist()

In [22]:
p1

[' \n \nPage 1 of 13 \n \nDisability Policy  \n \nIntroduction  \n \nThis policy forms part of the School’s overall commitment to ensuring equality for its diverse staff and \nstudent body.',
 'It outlines the School’s duties and the options available to both students and staff with \ndisabilities, including mental health disabilit ies, in seeking support or adjustments to their studying and \nworking environment; and it provides a framework to support the ongoing development of an inclusive, safe, \nenabling and supportive environment for all of the School’s staff, students, and visitor s, regardless of \ndisability.',
 'This policy should be read in conjunction with the School’s Equality, Diversity and Inclusion Policy .',
 ' \n \nScope  \n \nThis policy applies to all LIS staff and students.',
 'Where relevant, the policy also applies to prospective staff, \nprospective students, and visitors.',
 ' \n \nLegislative context: Equality Act 2010  \n \nAs a disability is a protected char

In [23]:
# define a function to remove punctuation and numbers from a string
def remove_punctuation_and_numbers(text):
    # define a regular expression pattern to match punctuation characters and numbers
    pattern = r'[^A-Za-z\s]'
    # replace punctuation characters and numbers with an empty string
    cleaned_text = re.sub(pattern, '', text)
    return cleaned_text

# apply the previously created function to each string in the list
cleaned_p1 = [remove_punctuation_and_numbers(string) for string in p1]

# display the cleaned list of strings
print(cleaned_p1)

[' \n \nPage  of  \n \nDisability Policy  \n \nIntroduction  \n \nThis policy forms part of the Schools overall commitment to ensuring equality for its diverse staff and \nstudent body', 'It outlines the Schools duties and the options available to both students and staff with \ndisabilities including mental health disabilit ies in seeking support or adjustments to their studying and \nworking environment and it provides a framework to support the ongoing development of an inclusive safe \nenabling and supportive environment for all of the Schools staff students and visitor s regardless of \ndisability', 'This policy should be read in conjunction with the Schools Equality Diversity and Inclusion Policy ', ' \n \nScope  \n \nThis policy applies to all LIS staff and students', 'Where relevant the policy also applies to prospective staff \nprospective students and visitors', ' \n \nLegislative context Equality Act   \n \nAs a disability is a protected characteristic under the Equality Act 

In [24]:
# tokenize each string in the list of strings
tokenized_p1 = [word_tokenize(string) for string in cleaned_p1]

# display the tokenized list of strings
print(tokenized_p1)

[['Page', 'of', 'Disability', 'Policy', 'Introduction', 'This', 'policy', 'forms', 'part', 'of', 'the', 'Schools', 'overall', 'commitment', 'to', 'ensuring', 'equality', 'for', 'its', 'diverse', 'staff', 'and', 'student', 'body'], ['It', 'outlines', 'the', 'Schools', 'duties', 'and', 'the', 'options', 'available', 'to', 'both', 'students', 'and', 'staff', 'with', 'disabilities', 'including', 'mental', 'health', 'disabilit', 'ies', 'in', 'seeking', 'support', 'or', 'adjustments', 'to', 'their', 'studying', 'and', 'working', 'environment', 'and', 'it', 'provides', 'a', 'framework', 'to', 'support', 'the', 'ongoing', 'development', 'of', 'an', 'inclusive', 'safe', 'enabling', 'and', 'supportive', 'environment', 'for', 'all', 'of', 'the', 'Schools', 'staff', 'students', 'and', 'visitor', 's', 'regardless', 'of', 'disability'], ['This', 'policy', 'should', 'be', 'read', 'in', 'conjunction', 'with', 'the', 'Schools', 'Equality', 'Diversity', 'and', 'Inclusion', 'Policy'], ['Scope', 'This', '

In [25]:
# as some strings are now empty, remove empty strings from a list of strings
cleaned_list_of_strings = [string for string in tokenized_p1 if string]

# and just in case that doesn't work,, remove empty strings from a list of lists of strings
cleaned_tokenized_p1 = [[token for token in tokens if token] for tokens in tokenized_p1]

In [26]:
cleaned_tokenized_p1

[['Page',
  'of',
  'Disability',
  'Policy',
  'Introduction',
  'This',
  'policy',
  'forms',
  'part',
  'of',
  'the',
  'Schools',
  'overall',
  'commitment',
  'to',
  'ensuring',
  'equality',
  'for',
  'its',
  'diverse',
  'staff',
  'and',
  'student',
  'body'],
 ['It',
  'outlines',
  'the',
  'Schools',
  'duties',
  'and',
  'the',
  'options',
  'available',
  'to',
  'both',
  'students',
  'and',
  'staff',
  'with',
  'disabilities',
  'including',
  'mental',
  'health',
  'disabilit',
  'ies',
  'in',
  'seeking',
  'support',
  'or',
  'adjustments',
  'to',
  'their',
  'studying',
  'and',
  'working',
  'environment',
  'and',
  'it',
  'provides',
  'a',
  'framework',
  'to',
  'support',
  'the',
  'ongoing',
  'development',
  'of',
  'an',
  'inclusive',
  'safe',
  'enabling',
  'and',
  'supportive',
  'environment',
  'for',
  'all',
  'of',
  'the',
  'Schools',
  'staff',
  'students',
  'and',
  'visitor',
  's',
  'regardless',
  'of',
  'disabili

In [27]:
#lemmatize
postlemma = []

for i in cleaned_tokenized_p1: 
    postlemma.append([lemmatizer.lemmatize(j) for j in i])

In [28]:
postlemma

[['Page',
  'of',
  'Disability',
  'Policy',
  'Introduction',
  'This',
  'policy',
  'form',
  'part',
  'of',
  'the',
  'Schools',
  'overall',
  'commitment',
  'to',
  'ensuring',
  'equality',
  'for',
  'it',
  'diverse',
  'staff',
  'and',
  'student',
  'body'],
 ['It',
  'outline',
  'the',
  'Schools',
  'duty',
  'and',
  'the',
  'option',
  'available',
  'to',
  'both',
  'student',
  'and',
  'staff',
  'with',
  'disability',
  'including',
  'mental',
  'health',
  'disabilit',
  'y',
  'in',
  'seeking',
  'support',
  'or',
  'adjustment',
  'to',
  'their',
  'studying',
  'and',
  'working',
  'environment',
  'and',
  'it',
  'provides',
  'a',
  'framework',
  'to',
  'support',
  'the',
  'ongoing',
  'development',
  'of',
  'an',
  'inclusive',
  'safe',
  'enabling',
  'and',
  'supportive',
  'environment',
  'for',
  'all',
  'of',
  'the',
  'Schools',
  'staff',
  'student',
  'and',
  'visitor',
  's',
  'regardless',
  'of',
  'disability'],
 ['This

In [29]:
#lower all cases
postfin1 = [[j.lower() for j in i] for i in postlemma]

In [30]:
postfin1

[['page',
  'of',
  'disability',
  'policy',
  'introduction',
  'this',
  'policy',
  'form',
  'part',
  'of',
  'the',
  'schools',
  'overall',
  'commitment',
  'to',
  'ensuring',
  'equality',
  'for',
  'it',
  'diverse',
  'staff',
  'and',
  'student',
  'body'],
 ['it',
  'outline',
  'the',
  'schools',
  'duty',
  'and',
  'the',
  'option',
  'available',
  'to',
  'both',
  'student',
  'and',
  'staff',
  'with',
  'disability',
  'including',
  'mental',
  'health',
  'disabilit',
  'y',
  'in',
  'seeking',
  'support',
  'or',
  'adjustment',
  'to',
  'their',
  'studying',
  'and',
  'working',
  'environment',
  'and',
  'it',
  'provides',
  'a',
  'framework',
  'to',
  'support',
  'the',
  'ongoing',
  'development',
  'of',
  'an',
  'inclusive',
  'safe',
  'enabling',
  'and',
  'supportive',
  'environment',
  'for',
  'all',
  'of',
  'the',
  'schools',
  'staff',
  'student',
  'and',
  'visitor',
  's',
  'regardless',
  'of',
  'disability'],
 ['this

In [31]:
# replace the old comments with the new pre-processed comments
data = {'Sentence': postfin1}
p1 = pd.DataFrame(data)

In [32]:
p1

,Sentence
0,"[page, of, disability, policy, introduction, t..."
1,"[it, outline, the, schools, duty, and, the, op..."
2,"[this, policy, should, be, read, in, conjuncti..."
3,"[scope, this, policy, applies, to, all, lis, s..."
4,"[where, relevant, the, policy, also, applies, ..."
...,...
145,"[clarified, the, role, of, the, equality, dive..."
146,"[included, context, for, how, the, policy, wa,..."
147,"[e, xpanded, detail, on, reasonable, adjustmen..."
148,"[hannah, kohler, director, of, admissions, and..."


In [33]:
# set display options to show all columns and rows in the dataframe so i can quickly scan through
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

# display the dataframe
print(p1)

                                              Sentence
0    [page, of, disability, policy, introduction, t...
1    [it, outline, the, schools, duty, and, the, op...
2    [this, policy, should, be, read, in, conjuncti...
3    [scope, this, policy, applies, to, all, lis, s...
4    [where, relevant, the, policy, also, applies, ...
5    [legislative, context, equality, act, as, a, d...
6    [advance, equality, of, opportunity, between, ...
7    [foster, good, relation, between, person, who,...
8    [public, body, are, required, to, integrate, c...
9    [this, enables, the, school, to, reasonably, a...
10   [development, of, this, policy, this, policy, ...
11   [when, we, have, student, enrolled, in, our, p...
12   [definitions, disability, for, the, purpose, o...
13   [page, of, while, understanding, the, medical,...
14   [barriers, can, be, in, the, physical, environ...
15   [the, school, aim, to, address, all, of, these...
16   [reasonable, adjustment, a, reasonable, adjust...
17   [reas

In [34]:
# Doing so helped me notice that there are a bunch of empty lists, so we will remove those now
# find rows with empty lists in the 'Sentence' column
rows_with_empty_lists = p1[p1['Sentence'].apply(len) == 0]

# display rows with empty lists
print(rows_with_empty_lists)

Empty DataFrame
Columns: [Sentence]
Index: []


In [35]:
# remove rows with empty lists in the 'Sentence' column
p1 = p1[p1['Sentence'].apply(len) > 0]

# reset index
p1.reset_index(drop=True, inplace=True)

# display the modified DataFrame
print(p1)

                                              Sentence
0    [page, of, disability, policy, introduction, t...
1    [it, outline, the, schools, duty, and, the, op...
2    [this, policy, should, be, read, in, conjuncti...
3    [scope, this, policy, applies, to, all, lis, s...
4    [where, relevant, the, policy, also, applies, ...
5    [legislative, context, equality, act, as, a, d...
6    [advance, equality, of, opportunity, between, ...
7    [foster, good, relation, between, person, who,...
8    [public, body, are, required, to, integrate, c...
9    [this, enables, the, school, to, reasonably, a...
10   [development, of, this, policy, this, policy, ...
11   [when, we, have, student, enrolled, in, our, p...
12   [definitions, disability, for, the, purpose, o...
13   [page, of, while, understanding, the, medical,...
14   [barriers, can, be, in, the, physical, environ...
15   [the, school, aim, to, address, all, of, these...
16   [reasonable, adjustment, a, reasonable, adjust...
17   [reas

In [36]:
# just to check it is still all good in this format too (can't be too careful)
p1

,Sentence
0,"[page, of, disability, policy, introduction, t..."
1,"[it, outline, the, schools, duty, and, the, op..."
2,"[this, policy, should, be, read, in, conjuncti..."
3,"[scope, this, policy, applies, to, all, lis, s..."
4,"[where, relevant, the, policy, also, applies, ..."
5,"[legislative, context, equality, act, as, a, d..."
6,"[advance, equality, of, opportunity, between, ..."
7,"[foster, good, relation, between, person, who,..."
8,"[public, body, are, required, to, integrate, c..."
9,"[this, enables, the, school, to, reasonably, a..."


In [37]:
# save the dataset to be able to use in analysis

#export to excel file
p1.to_excel('p2.xlsx')

#export to csv file
p1.to_csv('p2.csv', index=False)